# 1.

In [2]:
from math import sin
import math
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
import numpy as np

global B 
global M
global L
G = 9.81

In [3]:
def f(x, y, u):
    return -((B/M) * u) - ((G/L) * sin(y))

In [4]:
def calcular_yn_siguiente(yn, h, m1, m2, m3, m4):
    return yn + (h/6) * (m1 + 2 * m2 + 2 * m3 + m4)

def calcular_un_siguiente(un, h,k1, k2, k3, k4):
    return un + (h/6) * (k1 + 2 * k2 + 2 * k3 + k4)

In [5]:
def calcular_m_y_k(xn, yn, un, f, h):
    m1 = un
    k1 = f(xn, yn ,un)
    m2 = un + (h * k1) / 2
    k2 = f(xn + h/2, yn + (h * m1) / 2, un + (h * k1) / 2)
    m3 = un + (h * k2) / 2
    k3 = f(xn + h/2, yn + (h * m2) / 2, un + (h * k2) / 2)
    m4 = un + h * k3
    k4 = f (xn + h, yn + (h * m3), un + (h * k3))
    return {'m1': m1, 'm2': m2, 'm3': m3, 'm4': m4, 'k1': k1, 'k2': k2, 'k3': k3, 'k4': k4}

In [6]:
def resolver_ecuacion_diferencial(f, x_inicial, x_final, h, y0, yy0):
    resultado = {'t[s]': [], 'θ[Rad]': [], 'θʹ[Rad/s]': []}
    xn = x_inicial
    yn = math.radians(y0)
    un = math.radians(yy0)
    while xn <= x_final:
        resultado['t[s]'].append(xn)
        resultado['θ[Rad]'].append(yn)
        resultado['θʹ[Rad/s]'].append(un)       
        m_y_k = calcular_m_y_k(xn, yn, un, f, h)
        yn_sig = calcular_yn_siguiente(yn, h, m_y_k['m1'], m_y_k['m2'], m_y_k['m3'], m_y_k['m4'])
        un_sig = calcular_un_siguiente(un, h, m_y_k['k1'], m_y_k['k2'], m_y_k['k3'], m_y_k['k4'])
        yn = yn_sig
        un = un_sig
        xn = xn + h
    return resultado

In [7]:
def obtener_tabla_reducida(resultados_de_ecuacion_diferencial):
  largo_de_listas_en_tabla = len(resultados_de_ecuacion_diferencial['t[s]'])

  if largo_de_listas_en_tabla <= 10:
    return resultados_de_ecuacion_diferencial
  else:
    #agrego primeros 5 y ultimos 5
    tabla_reducida = {}
    tabla_reducida['t[s]'] = resultados_de_ecuacion_diferencial['t[s]'][0:6]
    tabla_reducida['t[s]'].extend(resultados_de_ecuacion_diferencial['t[s]'][largo_de_listas_en_tabla - 6:])
    tabla_reducida['θ[Rad]'] = resultados_de_ecuacion_diferencial['θ[Rad]'][0:6]
    tabla_reducida['θ[Rad]'].extend(resultados_de_ecuacion_diferencial['θ[Rad]'][largo_de_listas_en_tabla - 6:])
    tabla_reducida['θʹ[Rad/s]'] = resultados_de_ecuacion_diferencial['θʹ[Rad/s]'][0:6]
    tabla_reducida['θʹ[Rad/s]'].extend(resultados_de_ecuacion_diferencial['θʹ[Rad/s]'][largo_de_listas_en_tabla - 6:])
    return tabla_reducida

In [8]:
def settear_constantes(m, l, b):
  #cambio las variable globales asi no es necesario pasarle como parametro a la funcion f las constantes m,l y b
  #cada vez que se la llama
  global M
  global B
  global L
  M = m
  B = b
  L = l

In [9]:
def pasar_radianes_a_grados(datos):
  #pasa todos los datos de una lista que estan en radianes a grados
  resultado = []
  for dato in datos:
    resultado.append(math.degrees(dato))
  return resultado

In [10]:
def resolver_pendulo_rg4(m, l, b, h, y0, yy0):
  settear_constantes(m, l, b)
  df = pd.DataFrame(obtener_tabla_reducida(resolver_ecuacion_diferencial(f, 0, 20, h, y0, yy0)))
  print(tabulate(df, showindex= False,headers = 'keys', tablefmt="fancy_grid", floatfmt=(".1f", ".13f", ".13f")))



# 2.

## a) Sistema no amortiguado

In [ ]:
resolver_pendulo_rg4(1, 1, 1, 0.2, 30, 0)

In [ ]:
settear_constantes(1, 1, 0)
info = resolver_ecuacion_diferencial(f, 0, 20, 0.2, 30, 0)
x = np.arange(0, 20.2, 0.2)
y_angulo = pasar_radianes_a_grados(info['θ[Rad]'])
y_vel_angular = pasar_radianes_a_grados(info['θʹ[Rad/s]'])
plt.plot(x, y_angulo, color = 'r', label = 'Angulo')
plt.plot(x, y_vel_angular, color = 'b', label = 'Velocidad angular')
plt.ylim(-100, 100)
plt.xlim(0, 20)
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.legend()
plt.grid()
plt.xlabel('t [s]')
plt.ylabel('θ [°]     θʹ [°/s]')
plt.show()

## b) Sistema con amortiguamiento subcrítico

In [ ]:
resolver_pendulo_rg4(1, 1, 0.5, 0.2, 30, 100)

In [ ]:
settear_constantes(1, 1, 0.5)
info = resolver_ecuacion_diferencial(f, 0, 20, 0.2, 30, 100)
x = np.arange(0, 20.2, 0.2)
y_angulo = pasar_radianes_a_grados(info['θ[Rad]'])
y_vel_angular = pasar_radianes_a_grados(info['θʹ[Rad/s]'])
plt.plot(x, y_angulo, color = 'r', label = 'Angulo')
plt.plot(x, y_vel_angular, color = 'b', label = 'Velocidad angular')
plt.ylim(-150, 150)
plt.xlim(0, 20)
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.legend()
plt.grid()
plt.xlabel('t [s]')
plt.ylabel('θ [°]     θʹ [°/s]')
plt.show()